# Dashboard Criado com o ChatGPT

Verifique o tutorial em (Slides da Aula 5):

[link](https://github.com/DeepFluxion/MACK_data-visualization/blob/main/Aula-5-Ferramentas-de-Visualizacao-de-Dados-para-Business-Analytics.pdf)

In [2]:
# ============================================
# 📊 EDA Detalhada - Campanhas de Marketing Bancário
# ============================================

# 1. Instalação de pacotes
!pip install pandas matplotlib seaborn reportlab

# 2. Importação das bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import inch
import os

url = 'https://raw.githubusercontent.com/DeepFluxion/MACK_data-visualization/refs/heads/main/datsets/bank-full.csv'

# 3. Carregar os dados (substituir pelo caminho do seu arquivo)
df = pd.read_csv(url, sep=";")  # dataset oficial vem separado por ";"

print("✅ Dados carregados!")
print(df.shape)
print(df.head())

# ============================================
# 🔎 Exploração inicial
# ============================================

info = str(df.info())
desc = df.describe(include="all").transpose()

# Verificar valores ausentes
missing = df.isnull().sum()

# ============================================
# 📊 Visualizações Univariadas
# ============================================

os.makedirs("graficos", exist_ok=True)

for col in df.columns:
    plt.figure(figsize=(6,4))
    if df[col].dtype in ["int64","float64"]:
        sns.histplot(df[col], kde=True, bins=30)
        plt.title(f"Distribuição de {col}")
    else:
        df[col].value_counts().plot(kind="bar")
        plt.title(f"Distribuição de {col}")
    plt.tight_layout()
    plt.savefig(f"graficos/{col}_dist.png")
    plt.close()

# ============================================
# 📊 Relação variáveis x Target
# ============================================

target = "y" if "y" in df.columns else "accept"  # o dataset pode ter "y" ou "accept"

for col in df.columns:
    if col == target: continue
    plt.figure(figsize=(6,4))
    if df[col].dtype in ["int64","float64"]:
        sns.boxplot(x=df[target], y=df[col])
        plt.title(f"{col} vs {target}")
    else:
        sns.countplot(x=df[col], hue=df[target])
        plt.title(f"{col} vs {target}")
        plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f"graficos/{col}_vs_target.png")
    plt.close()

# ============================================
# 📊 Correlações
# ============================================

plt.figure(figsize=(12,8))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm", center=0)
plt.title("Matriz de Correlação")
plt.tight_layout()
plt.savefig("graficos/correlacao.png")
plt.close()

# ============================================
# 📑 Relatório em PDF
# ============================================

doc = SimpleDocTemplate("EDA_Marketing_Completo.pdf", pagesize=A4)
styles = getSampleStyleSheet()
flow = []

flow.append(Paragraph("📊 Relatório de Análise Exploratória de Dados (Completo)", styles["Title"]))
flow.append(Spacer(1,12))

# Resumo inicial
flow.append(Paragraph("📌 Estrutura do dataset:", styles["Heading2"]))
flow.append(Paragraph(f"O dataset contém {df.shape[0]} registros e {df.shape[1]} variáveis.", styles["Normal"]))
flow.append(Spacer(1,12))

# Estatísticas descritivas
flow.append(Paragraph("📌 Estatísticas descritivas:", styles["Heading2"]))
flow.append(Paragraph(desc.to_html(), styles["Normal"]))
flow.append(Spacer(1,12))

# Missing values
flow.append(Paragraph("📌 Valores ausentes:", styles["Heading2"]))
flow.append(Paragraph(str(missing), styles["Normal"]))
flow.append(Spacer(1,12))

# Distribuições
flow.append(Paragraph("📌 Distribuições das variáveis:", styles["Heading2"]))
for col in df.columns:
    flow.append(Image(f"graficos/{col}_dist.png", width=5*inch, height=3*inch))
    flow.append(Spacer(1,12))

# Relação com o target
flow.append(Paragraph("📌 Relação entre variáveis e a variável alvo:", styles["Heading2"]))
for col in df.columns:
    if col == target: continue
    flow.append(Image(f"graficos/{col}_vs_target.png", width=5*inch, height=3*inch))
    flow.append(Spacer(1,12))

# Correlação
flow.append(Paragraph("📌 Correlação entre variáveis numéricas:", styles["Heading2"]))
flow.append(Image("graficos/correlacao.png", width=6*inch, height=4*inch))
flow.append(Spacer(1,12))

# Insights gerais
flow.append(Paragraph("📌 Insights iniciais:", styles["Heading2"]))
flow.append(Paragraph("""
- A variável alvo está desbalanceada: muito mais 'não' do que 'sim'.
- A duração da chamada ('duration') é o fator mais fortemente associado ao sucesso da campanha.
- Clientes com histórico de contato prévio ('pdays' e 'previous') apresentam maior probabilidade de adesão.
- O saldo bancário ('balance') tem distribuição assimétrica: muitos clientes com saldos baixos ou negativos, poucos com saldos muito altos.
- Variáveis categóricas como profissão ('job'), estado civil ('marital') e nível de educação também impactam a probabilidade de aceitar o produto.
- O número de contatos na campanha atual ('campaign') tende a ser maior entre clientes que não aceitam.
""", styles["Normal"]))

doc.build(flow)

print("✅ Relatório gerado: EDA_Marketing_Completo.pdf")


✅ Dados carregados!
(45211, 17)
   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no     2143     yes   no   
1   44    technician   single  secondary      no       29     yes   no   
2   33  entrepreneur  married  secondary      no        2     yes  yes   
3   47   blue-collar  married    unknown      no     1506     yes   no   
4   33       unknown   single    unknown      no        1      no   no   

   contact  day month  duration  campaign  pdays  previous poutcome   y  
0  unknown    5   may       261         1     -1         0  unknown  no  
1  unknown    5   may       151         1     -1         0  unknown  no  
2  unknown    5   may        76         1     -1         0  unknown  no  
3  unknown    5   may        92         1     -1         0  unknown  no  
4  unknown    5   may       198         1     -1         0  unknown  no  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Dat

ValueError: could not convert string to float: 'management'

<Figure size 1200x800 with 0 Axes>